In [6]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
TABLE = 'Deadline'

In [7]:
url = 'https://www.monroecollege.edu/academics/academic-calendar-and-resources'
page = requests.get(url)

with open(TABLE + '.html', 'w', encoding='utf-8') as file:
    file.write(page.text)

In [8]:
with open(TABLE + '.html', 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

In [9]:
h2_elements = soup.find_all('h2')
h2_elements

[<h2 class="visually-hidden" id="block-mainnavigation-menu">Main navigation</h2>,
 <h2 class="visually-hidden" id="block-submainnavigation-menu">Sub main navigation</h2>,
 <h2 class="visually-hidden" id="dialog-title-1">Enter a Search Term</h2>,
 <h2 class="visually-hidden" id="block-mobile-menu-menu">Mobile Main navigation</h2>,
 <h2 class="visually-hidden" id="block-mobile-sub-menu-menu">Mobile Sub main navigation</h2>,
 <h2 class="visually-hidden" id="dialog-title-2">Enter a Search Term</h2>,
 <h2>Winter 2024 Academic Calendar</h2>,
 <h2>Spring 2024 Academic Calendar</h2>,
 <h2>August Bridge / Fall 2024 Academic Calendar</h2>,
 <h2>Winter 2025 Academic Calendar</h2>,
 <h2>Spring 2025 Academic Calendar</h2>,
 <h2>Course Catalog</h2>,
 <h2>Transcript Requests</h2>,
 <h2>Academic Tutoring</h2>,
 <h2>Academic Centers</h2>,
 <h2>Library Services</h2>,
 <h2>Questions?</h2>,
 <h2 class="visually-hidden">Site Footer</h2>]

In [10]:
df_list = []

h2_elements = soup.find_all('h2')

for h2 in h2_elements:
    h2_text = h2.get_text()
    
    if 'Calendar' in h2_text:
        subsequent_siblings = list(h2.find_all_next())
        
        current_p = None
        for sibling in subsequent_siblings:
            if sibling.name == 'h2':
                break
            if sibling.name == 'p' and ('Calendar' in sibling.get_text()):
                current_p = sibling.get_text().strip()
            if sibling.name == 'table':
                temp_df = pd.read_html(str(sibling))[0]
                temp_df['Term'] = h2_text
                temp_df['Sub-Term'] = current_p
                df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)
df.rename(columns={0: "Date", 1: "Day", 2: "Information"}, inplace=True)
df['Date'] = df[['Day', 'Date']].apply(lambda row: ', '.join(row.values), axis=1)
df.drop('Day', axis=1, inplace=True)
# df.reset_index(drop=True, inplace=True)
df = df[df.columns[[2, 3, 0, 1]]]
df.to_csv('deadline.csv')

/var/folders/jl/k_qw56hn2dx0cbwjpqwf8k3m0000gn/T/ipykernel_47338/1443221669.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(sibling))[0]
/var/folders/jl/k_qw56hn2dx0cbwjpqwf8k3m0000gn/T/ipykernel_47338/1443221669.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(sibling))[0]
/var/folders/jl/k_qw56hn2dx0cbwjpqwf8k3m0000gn/T/ipykernel_47338/1443221669.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(sibling))[0]
/var/folders/jl/k_qw56hn2dx0cbwjpqwf8k3m0000gn/T/ipykernel_47338/1443221669.py:18: FutureWarning: Passing literal html

In [11]:
command = f"jupyter nbconvert --to script {TABLE}"
os.system(command)

[NbConvertApp] Converting notebook Deadline.ipynb to script
[NbConvertApp] Writing 1757 bytes to Deadline.py


0